In [ ]:
from pathlib import Path
import os
import geopandas as gpd

from mclights.get_data import get_data_osmx_or_local

project_dir = Path(os.getcwd()).parent
data_dir = project_dir / "data"
raw_data_dir = project_dir / "raw_data"

voivodeships_names = [
    "Województwo Dolnośląskie",
    "Województwo Kujawsko-Pomorskie",
    "Województwo Lubelskie",
    "Województwo Lubuskie",
    "Województwo Łódzkie",
    "Województwo Małopolskie",
    "Województwo Mazowieckie",
    "Województwo Opolskie",
    "Województwo Podkarpackie",
    "Województwo Podlaskie",
    "Województwo Pomorskie",
    "Województwo Śląskie",
    "Województwo Świętokrzyskie",
    "Województwo Warmińsko-Mazurskie",
    "Województwo Wielkopolskie",
    "Województwo Zachodniopomorskie"
]

In [ ]:
tags={"boundary": "administrative","admin_level": "4"}
voivodeships = get_data_osmx_or_local(raw_data_dir / "voivodeships.gpkg", tags)

voivodeships = voivodeships.loc[voivodeships["element"] == "relation", ["name", "geometry"]]
voivodeships["name"] = voivodeships["name"].str.title()
voivodeships = voivodeships[voivodeships["name"].isin(voivodeships_names)]
voivodeships = voivodeships[~voivodeships["name"].str.contains("Urząd")]
voivodeships = voivodeships.drop_duplicates()

voivodeships_path = data_dir / "voivodeships.gpkg"
voivodeships.to_file(voivodeships_path, layer=voivodeships_path.stem, driver="GPKG")

In [ ]:
cities = get_data_osmx_or_local(raw_data_dir / "cities_raw.gpkg", {"place": ["city", "town", "village"]})

cities = cities.loc[
    cities["element"] == "node",
    ["geometry", "name", "place", "population", "name:pl", "wikipedia"]
]
cities["name"] = (
    cities["name"]
        .fillna(cities["name:pl"])
        .fillna(cities["wikipedia"].str.extract(r"^pl:(.*?)\s*\(", expand=False))
)
cities = cities.dropna(subset=["name"])
cities = cities.drop(columns=["name:pl", "wikipedia"])

cities = gpd.sjoin(
    cities,
    voivodeships.rename(columns={"name": "voivodeship"}),
    how="left",
    predicate="within",
).drop(columns="index_right")

cities_path = data_dir / "cities.gpkg"
cities.to_file(cities_path, layer=cities_path.stem, driver="GPKG")

In [ ]:
tags={"highway": "motorway"}
motorways = get_data_osmx_or_local(raw_data_dir / "motorways.gpkg", tags)

# motorways_path = data_dir / "motorways.gpkg"
# motorways.to_file(motorways_path, layer=motorways_path.stem, driver="GPKG")

In [ ]:
voivodenships_file_path = data_dir / "voivodeships.gpkg"
voivodenships = gpd.read_file(voivodenships_file_path, layer=voivodenships_file_path.stem)

In [ ]:
motorways[motorways["element"] == "way"]

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))

motorways.loc[""].plot(ax=ax, facecolor="None")

plt.show()